In [1]:
import pandas as pd
import numpy as np
import os

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf
import repofuncs.reapmonthlyfuncs as rmf

import warnings
warnings.filterwarnings('ignore')

path_to_matrix = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall"
path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)
mf.read_directory()

report_data = pd.read_csv('Netcall Dashboard Data.csv',low_memory=False)
report_data = mf.strip_dataframe(report_data)

lookup_data = pd.read_excel('Queue to Group Lookup.xlsx')
lookup_data = mf.strip_dataframe(lookup_data)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data
DataFrame: files
Number of records: 15 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,id_check.csv
1,Interactions Apr 2025 (excl Cust Serv)
2,Interactions August 2025 (excl Cust Serv)
3,Interactions Feb 2023 - Apr 2024 (excl Cust Serv)
4,Interactions Feb 2025 (excl Cust Serv)
5,Interactions Jan 2022 - Dec 2022 (excl Cust Serv)
6,Interactions Jan 2024 - Jun 2024 (excl Cust Serv)
7,Interactions July 2025 (excl Cust Serv)
8,Interactions Jun 24 - Jan 25 (excl Cust Serv)
9,Interactions Mar 2025 (excl Cust Serv)


In [2]:
lookup_columns = {
    'Group':'group',
    'Group Name':'groupname',
    'Queue Summary':'queuesummary',
    'Queue':'queue'
}

groups_to_order = {
    "Contact Centre":0,
    "Accessible Transport":1,
    "Benefits":2,
    "Complaints":3,
    "Cost of Living":4,
    "Council Tax":5,
    "Electoral Services":6,
    "Enforcement and Licensing":7,
    "Other":8,
    "Parking":9,
    "Registrars":10,
    "Sports and Leisure":11,
    "Switchboard":12,
    "Waste":13,
    "Repair CSC":14,
    "Parking Services":15,
    "Rent Income Services":16,
    "Housing Advice":17,
    "Adult Social Care":18,
    "Childrens Social Care":19,
    "Other Groups":20
}

columns_to_rename = {
    "group":"Group",
    "number_of_interactions":"Total Calls",
    "agent_handling_time":"Total Handling Time (Days)",
    "abandoned":"% Abandoned",
    "answered":"% Answered",
    "callback":"% Callback",
    "redirected":"% Redirected",
    "transferred":"% Transferred",
    r"% change 12 month average":r"% Change 12 Month Average",
    r"% change same month last year":r"% Change Same Month Last Year",
    "average_handling_time":"Average Handling Time (Minutes)",
    "average_queue_time":"Average Queue Time (Minutes)"
}

columns_to_order = [
    "Group",
    "Total Calls",
    "% Change Same Month Last Year",
    "% Change 12 Month Average",
    "% Answered",
    "% Abandoned",
    "% Transferred",
    "% Callback",
    "% Redirected",
    "Total Handling Time (Days)",
    "Average Handling Time (Minutes)",
    "Average Queue Time (Minutes)"
]

In [3]:
columns_to_cummulate = [
    'number_of_interactions',
    'agent_handling_time',
    'queue_time',
    'abandoned',
    'answered',
    'callback',
    'redirected',
    'transferred'
]

rows_to_order = {
    "number_of_interactions":0,
    "answered":1,
    "average_queue_time":2,
    "agent_handling_time":3,
    "average_handling_time":4,
    "abandoned":5,
    "transferred":6,
    "redirected":7,
    "callback":8
}

rows_to_rename = {
    "number_of_interactions":"Total Calls Offered",
    "answered":"% Calls Answered",
    "average_queue_time":"Average Queue Time (mins)",
    "agent_handling_time":"Total Handling Time (days)",
    "average_handling_time":"Average Handling Time (mins)",
    "abandoned":"% Calls Abandoned",
    "transferred":"% Calls Transferred",
    "redirected":"% Calls Redirected",
    "callback":"% Calls Callback"
}

date_dic = rmf.obtain_relevant_dates(report_data, 'dateofqueue')

Aggregating Data

In [4]:
report_df = report_data.copy()
report_df['monthofqueue'] = pd.to_datetime(report_df['dateofqueue']).dt.strftime("%Y-%m")

report_df = report_df.groupby(['direction','type','monthofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('number_of_interactions', 'sum'),
    wrap_upduration = ('wrap_upduration','sum'),
    queue_time = ('queue_time','sum'),
    duration = ('duration','sum'),
    talk_time = ('talk_time','sum'),
    hold_time = ('hold_time','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
).reset_index()

report_df = report_df[
    (report_df['direction'] == 'Inbound') &
    (report_df['type'] == 'Call')
]

Merging the LookUp Data

In [5]:
report_df['composite_key'] = report_df['groupname'].str.lower() + ":" + report_df['queue'].str.lower()
lookup_data['composite_key'] = lookup_data['Group Name'].str.lower() + ":" + lookup_data['Queue'].str.lower()

report_df = report_df.merge(
    lookup_data, how='left',
    on='composite_key'
)

report_df.drop(
    columns=['groupname','queue', 'composite_key','direction','type'],
    inplace=True
)

report_df.rename(
    columns=lookup_columns,
    inplace=True
)

Pivoting the Call Result Data

In [6]:
results_data = report_df.pivot_table(
    index=['monthofqueue','group','queuesummary'],
    columns='result',
    values=['number_of_interactions'],
    aggfunc='sum'
)

results_data.columns = results_data.columns.droplevel(0)

results_data.columns = [column.lower() for column in results_data.columns]

results_data['total'] = results_data[results_data.columns.to_list()].sum(axis=1)

Aggregating the Main Report Data then Merging the Pivoted Call Results back to the Main Dataset

In [7]:
report_df = report_df.groupby(['monthofqueue','group','queuesummary']).agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum')
)

report_df = report_df.merge(
    results_data, how='left',
    left_index=True, right_index=True
).reset_index().sort_values(by=['monthofqueue','group','queuesummary'],ascending=False)

report_df['check'] = report_df['number_of_interactions'] == report_df['total']

if False not in report_df['check'].unique():
    report_df.drop(
        columns=['total','check','failed','other'],
        inplace=True
    )
else:
    raise ValueError('The number of interactions between the call results data and the main report data are not aligned!')

report_df['month'] = pd.to_datetime(report_df['monthofqueue'],format='%Y-%m').dt.month

report_df.sort_values(
    by=['group','queuesummary','monthofqueue'],
    inplace=True
)

In [8]:
telephony_data = report_df.groupby('monthofqueue').agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum'),
    abandoned = ('abandoned','sum'),
    answered = ('answered','sum'),
    callback = ('callback','sum'),
    redirected = ('redirected','sum'),
    transferred = ('transferred','sum'),
).reset_index()

telephony_data['queue_date'] = pd.to_datetime(telephony_data['monthofqueue'], format='%Y-%m')
telephony_data['month'] = telephony_data['queue_date'].dt.month
telephony_data['tax_year'] = telephony_data['queue_date'].apply(lambda x: x.year if x.month >= 4 else x.year-1)

for column in columns_to_cummulate:
    telephony_data[column] = telephony_data.groupby('tax_year')[column].cumsum()

telephony_data['average_handling_time'] = telephony_data['agent_handling_time'] / telephony_data['number_of_interactions']
telephony_data['average_queue_time'] = telephony_data['queue_time'] / telephony_data['number_of_interactions']


call_result_list = results_data.columns.tolist()
call_result_list = [x for x in call_result_list if x in telephony_data.columns.to_list()]
telephony_data[[f"{col}" for col in call_result_list]] = telephony_data[call_result_list].div(telephony_data["number_of_interactions"], axis=0)

telephony_data = telephony_data[telephony_data['month'] == date_dic['latest_date'].month-1]
telephony_data.drop(
    columns=['month','queue_date','queue_time','monthofqueue'],
    inplace=True
)

telephony_data.set_index(keys='tax_year', inplace=True)

telephony_data = telephony_data.tail(2)

telephony_data.rename(
    index = lambda year:f"{year} (Apr-{date_dic['prev_full_month_name']})",
    inplace=True
)

telephony_data = telephony_data.T

pct_change = rmf.compute_dynamic_change(telephony_data)

# Format agent_handling_time (row level now, not column)
if "agent_handling_time" in telephony_data.index:
    telephony_data.loc["agent_handling_time"] = telephony_data.loc["agent_handling_time"].apply(
        lambda x: f"{int(x // 86400)}d {int((x % 86400) // 3600)}h"
    )
 
# Format average handling & queue times (also row level now)
for row in ["average_handling_time", "average_queue_time"]:
    if row in telephony_data.index:
        telephony_data.loc[row] = telephony_data.loc[row].apply(
            lambda x: f"{int(x // 60)}m {x % 60:.0f}s"
        )

telephony_data = pd.concat(
    [telephony_data, pct_change],
    axis=1
)


telephony_data = telephony_data.replace([np.inf, -np.inf], np.nan).dropna(
    axis=1,
    how='all'
)

telephony_data.fillna('N/A', inplace=True)

telephony_data = telephony_data.reset_index().rename(
    columns={'index':'Metric'}
)
telephony_data['metric_order'] = telephony_data['Metric'].map(rows_to_order)

telephony_data.sort_values(
    by='metric_order',
    inplace=True
)

telephony_data['Metric'] = telephony_data['Metric'].map(rows_to_rename)
telephony_data.drop(
    columns='metric_order',
    inplace=True
)

In [9]:
mf.export_file(
    telephony_data,
    directory = path_to_matrix,
    df_name="Netcall's Telephony Matrix",
    file_type='xlsx',
    sheet_name='Sheet1'
)

Exporting Netcall's Telephony Matrix to XLSX...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\Netcall's Telephony Matrix.xlsx

Successfully exported Netcall's Telephony Matrix to XLSX


Transformations being applied to the Contact Centre Data

In [10]:
contact_centre_data = report_df[report_df['group'] == 'Customer Services']

contact_centre_data = rmf.calculate_pct(contact_centre_data, ['group','queuesummary'], 'number_of_interactions')

contact_centre_data = contact_centre_data[contact_centre_data['monthofqueue'] == date_dic['prev_full_month']]

Transformations being applied to the Service Data

In [11]:
service_data = report_df.groupby(['monthofqueue','month','group']).agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum'),
    abandoned = ('abandoned','sum'),
    answered = ('answered','sum'),
    callback = ('callback','sum'),
    redirected = ('redirected','sum'),
    transferred = ('transferred','sum'),
).reset_index()

service_data = rmf.calculate_pct(service_data, ['group'], 'number_of_interactions')

service_data = service_data[service_data['monthofqueue'] == date_dic['prev_full_month']]

Final Matrix Generation

In [12]:
group_telephony_data = pd.concat([service_data, contact_centre_data])
group_telephony_data.loc[group_telephony_data['queuesummary'].notna(), 'group'] = group_telephony_data['queuesummary']
group_telephony_data['average_handling_time'] = group_telephony_data['agent_handling_time'] / group_telephony_data['number_of_interactions']
group_telephony_data['average_queue_time'] = group_telephony_data['queue_time'] / group_telephony_data['number_of_interactions']

call_result_list = results_data.columns.tolist()
call_result_list = [x for x in call_result_list if x in group_telephony_data.columns.to_list()]
group_telephony_data[[f"{col}" for col in call_result_list]] = group_telephony_data[call_result_list].div(group_telephony_data["number_of_interactions"], axis=0)
group_telephony_data.fillna(0, inplace=True)

group_telephony_data['agent_handling_time'] = group_telephony_data['agent_handling_time'].apply(
    lambda x: f"{int(x // 86400)}d {int((x % 86400) // 3600)}h"
)

for column in ['average_handling_time','average_queue_time']:
    group_telephony_data[column] = group_telephony_data[column].apply(
        lambda x: f"{int(x // 60)}m {x % 60:.0f}s"
    )

group_telephony_data['group'] = group_telephony_data['group'].replace('Customer Services','Contact Centre')

group_telephony_data['group_order'] = group_telephony_data['group'].map(groups_to_order)
group_telephony_data.sort_values(
    by='group_order',
    inplace=True
)

group_telephony_data.drop(
    columns=['monthofqueue','month','queuesummary','group_order','rolling_12m_sum', 'queue_time'],
    inplace=True
)

group_telephony_data.rename(
    columns=columns_to_rename,
    inplace=True
)

group_telephony_data = group_telephony_data[columns_to_order]

mf.display(group_telephony_data, max_rows=True)

DataFrame: group_telephony_data
Number of records: 21 | Number of fields: 12
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,Group,Total Calls,% Change Same Month Last Year,% Change 12 Month Average,% Answered,% Abandoned,% Transferred,% Callback,% Redirected,Total Handling Time (Days),Average Handling Time (Minutes),Average Queue Time (Minutes)
322,Contact Centre,22069,0.079010,0.006379,0.565590,0.166795,0.116090,0.138973,0.012552,72d 14h,4m 44s,13m 25s
829,Accessible Transport,223,-0.766980,-0.377546,0.560538,0.147982,0.242152,0.035874,0.013453,0d 14h,4m 1s,5m 25s
830,Benefits,1556,-0.014566,-0.341285,0.481362,0.199871,0.070051,0.248715,0.000000,5d 19h,5m 22s,17m 43s
831,Complaints,354,0.388235,-0.176116,0.564972,0.135593,0.067797,0.000000,0.231638,1d 15h,6m 42s,8m 7s
832,Cost of Living,1074,-0.285429,-0.158076,0.940410,0.020484,0.033520,0.003724,0.001862,2d 23h,4m 0s,3m 59s
833,Council Tax,5869,0.128003,0.136713,0.488840,0.290680,0.047538,0.144999,0.027943,21d 18h,5m 20s,19m 59s
834,Electoral Services,116,-0.171429,-0.508534,0.870690,0.060345,0.068966,0.000000,0.000000,0d 9h,4m 46s,5m 46s
835,Enforcement and Licensing,491,-0.230408,0.053287,0.723014,0.073320,0.203666,0.000000,0.000000,1d 8h,4m 1s,5m 39s
836,Other,1777,0.275664,0.310758,0.758582,0.104671,0.126618,0.007316,0.002814,5d 10h,4m 24s,7m 54s
837,Parking,7131,0.607529,0.344361,0.539335,0.158463,0.049222,0.252980,0.000000,24d 21h,5m 2s,15m 32s


In [13]:
mf.export_file(
    group_telephony_data,
    directory = path_to_matrix,
    df_name="Netcall Latest Month's Matrix",
    file_type='xlsx'
)

Exporting Netcall Latest Month's Matrix to XLSX...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\Netcall Latest Month's Matrix.xlsx

Successfully exported Netcall Latest Month's Matrix to XLSX
